In [ ]:
import time
import pandas as pd
import pyodbc
from watchdog.observers import Observer
from watchdog.events import FileSystemEventHandler

# Rutas de archivos CSV
inventario_csv = r"C:\Users\beren\Desktop\HENRRY\G4_Inventory\AUTOMATIZACION\STAGING\Dim_Inventory.csv"
ventas_csv = r"C:\Users\beren\Desktop\HENRRY\G4_Inventory\AUTOMATIZACION\STAGING\Fact_Purchases_Final.csv"
compras_csv = r"C:\Users\beren\Desktop\HENRRY\G4_Inventory\AUTOMATIZACION\STAGING\Fact_Sales_Final.csv"

# Configuración de conexión a SQL Server
def connect_to_db():
    connection = pyodbc.connect(
        DRIVER='ODBC Driver 17 for SQL Server',
        SERVER='10.147.17.249\\SQLEXPRESS',
        DATABASE='prueba',
        UID='LARA',
        PWD='2307'
    )
    return connection

# Función para cargar el CSV y actualizar la base de datos
def update_database(file_path):
    # Conexión a SQL Server
    conn = connect_to_db()
    cursor = conn.cursor()

    # Detectar cuál archivo fue modificado y cargarlo
    if file_path == inventario_csv:
        df = pd.read_csv(inventario_csv)
        table_name = 'Dim_Inventory'
    elif file_path == ventas_csv:
        df = pd.read_csv(ventas_csv)
        table_name = 'Fact_Sales_Final'
    elif file_path == compras_csv:
        df = pd.read_csv(compras_csv)
        table_name = 'Fact_Purchases_Final'
    else:
        print(f"Archivo no reconocido: {file_path}")
        return

    # Insertar datos en la tabla correspondiente
    if table_name == 'Dim_Inventory':
        for index, row in df.iterrows():
            cursor.execute("""
                INSERT INTO Dim_Inventory (InventoryId, Store, City, Brand, Description, Size, OnHand_beg, Price_beg, StartDate, OnHand_end, Price_end, EndDate)
                VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
            """, row['InventoryId'], row['Store'], row['City'], row['Brand'], row['Description'], row['Size'], row['OnHand_beg'], row['Price_beg'], row['StartDate'], row['OnHand_end'], row['Price_end'], row['EndDate'])

    elif table_name == 'Fact_Sales_Final':
        for index, row in df.iterrows():
            cursor.execute("""
                INSERT INTO Fact_Sales_Final (Sales_ID, InventoryId, SalesQuantity, SalesDollars, SalesPrice, SalesDate, Volume, Classification, ExciseTax, VendorNo, VendorName)
                VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
            """, row['Sales_ID'], row['InventoryId'], row['SalesQuantity'], row['SalesDollars'], row['SalesPrice'], row['SalesDate'], row['Volume'], row['Classification'], row['ExciseTax'], row['VendorNo'], row['VendorName'])

    elif table_name == 'Fact_Purchases_Final':
        for index, row in df.iterrows():
            cursor.execute("""
                INSERT INTO Fact_Purchases_Final (Purchase_ID, InventoryId, VendorNumber, VendorName, PONumber, PODate, ReceivingDate, InvoiceDate, PayDate, PurchasePrice, Quantity, Dollars, Classification)
                VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
            """, row['Purchase_ID'], row['InventoryId'], row['VendorNumber'], row['VendorName'], row['PONumber'], row['PODate'], row['ReceivingDate'], row['InvoiceDate'], row['PayDate'], row['PurchasePrice'], row['Quantity'], row['Dollars'], row['Classification'])

    # Guardar los cambios
    conn.commit()
    cursor.close()
    conn.close()
    print(f"Base de datos actualizada con {file_path}")

# Clase para manejar eventos del sistema de archivos
class CSVChangeHandler(FileSystemEventHandler):
    def on_modified(self, event):
        if event.src_path.endswith('.csv'):
            print(f"Archivo modificado: {event.src_path}")
            update_database(event.src_path)

# Configurar el observador
if __name__ == "__main__":
    event_handler = CSVChangeHandler()
    observer = Observer()

    # Monitorear los tres archivos CSV
    observer.schedule(event_handler, path=inventario_csv, recursive=False)
    observer.schedule(event_handler, path=ventas_csv, recursive=False)
    observer.schedule(event_handler, path=compras_csv, recursive=False)

    # Iniciar el observador
    observer.start()
    print("Monitoreando cambios en archivos CSV...")

    try:
        while True:
            time.sleep(1)
    except KeyboardInterrupt:
        observer.stop()

    observer.join()


Monitoreando cambios en archivos CSV...
